In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load DATA


### Observations


In [2]:
bats_obs = xr.open_dataset("./products/Bats_obs.zarr", engine="zarr")
bats_obs

<xarray.Dataset> Size: 13kB
Dimensions:    (time: 538, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 4kB 1995-05-10 1995-05-11 ... 2022-12-13
Data variables:
    day        (time, latitude, longitude, layer) float64 4kB ...
    night      (time, latitude, longitude, layer) float64 4kB ...
Attributes:
    creation date:  2024-10-13T20:03:40.209673
    layer:          epipelagic
    station:        Bats

In [3]:
LATITUDE = bats_obs.latitude.data[0]
LONGITUDE = bats_obs.longitude.data[0]
# The observation data time range is starting too early. We will use the time range of the model data.
# START_TIME = pd.Timestamp(bats_obs.time.data[0]).strftime("%Y-%m-%d")
# END_TIME = pd.Timestamp(bats_obs.time.data[-1]).strftime("%Y-%m-%d")
START_TIME = "1998-01-01"
END_TIME = "2022-12-31"
DELTA_IMPORT = 0.5
DELTA_COMPUTE = 0.1

print(
    f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta import: {DELTA_IMPORT}"
)

Latitude: 31.6
Longitude: -64.2
Start Time: 1998-01-01
End Time: 2022-12-31
Delta import: 0.5


### CMEMS


In [4]:
FROM_API = True
path_cmems_product_bio = "..."
path_cmems_product_physic = "..."

In [5]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA_IMPORT,
    "maximum_latitude": LATITUDE + DELTA_IMPORT,
    "minimum_longitude": LONGITUDE - DELTA_IMPORT,
    "maximum_longitude": LONGITUDE + DELTA_IMPORT,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}

if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset(path_cmems_product_bio)
    physic = xr.load_dataset(path_cmems_product_physic)

INFO - 2024-11-19T14:38:27Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-11-19T14:38:27Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-11-19T14:38:27Z - The dataset cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i, version '202211', will be retired on the 2024-11-26T13:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-11-19T14:38:28Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-11-19T14:38:44Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-11-19T14:38:44Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-11-19T14:38:44Z - The dataset cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i, version '202211', will be retired on the 2024-11-26T13:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-11-19T14:38:45Z - Service was not

In [6]:
data = xr.merge([bio, physic])
lat_attrs, lon_attrs = data.latitude.attrs, data.longitude.attrs
data = data.sel(
    time=slice(START_TIME, END_TIME),
).sel(
    latitude=slice(LATITUDE - DELTA_COMPUTE, LATITUDE + DELTA_COMPUTE),
    longitude=slice(LONGITUDE - DELTA_COMPUTE, LONGITUDE + DELTA_COMPUTE),
)
data = data.mean(dim=["latitude", "longitude"], skipna=True)
data = data.expand_dims({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
data.latitude.attrs, data.longitude.attrs = lat_attrs, lon_attrs
data["T"].attrs["units"] = str(StandardUnitsLabels.temperature.units)
data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 9131, depth: 3)
Coordinates:
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
  * depth                (depth) int32 12B 1 2 3
Data variables: (12/13)
    mnkc_epi             (latitude, longitude, time) float64 73kB 0.7326 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 73kB 0.6836 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 73kB 1.118 ... 1...
    mnkc_mlmeso          (latitude, longitude, time) float64 73kB 0.5543 ... ...
    mnkc_mumeso          (latitude, longitude, time) float64 73kB 0.579 ... 0...
    mnkc_umeso           (latitude, longitude, time) float64 73kB 0.3336 ... ...
    ...                   ...
    zeu                  (latitude, longitude, time) float64 73kB 66.78 ... 7...
    zooc                 (latitude, longitude, time) float64 73kB 0.3742 ... ...
    T                    (latitude, longitude, time, depth) float64 219kB 20....
    U                    (latitude, longitude, time, depth) float64 219kB 0.0...
    V                    (latitude, longitude, time, depth) float64 219kB -0....
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 219kB 100...

## Gathering Data & Save


In [7]:
data.to_zarr("./products/Bats_cmems.zarr", mode="w")